In [1]:
#Importing Libraries
import pymongo as pym                                      #For Connectivity with Mongodb
import requests                                            #For Accessing the URL
import nltk                                                #For processing text from feeds
from bs4 import BeautifulSoup                              #For Web Scrapping
import urllib.request, urllib.parse, urllib.error          #For Parsing feeds
from newspaper import Article                              #For fetching Keyword
import schedule                                            #For scheduling function to run at specific interval
import time      
from datetime import date                                  #For formatting Pubdate or LastBuild Date 
import re                                                  #For removing html tags
import html                                                #For unescaping some html characters

In [ ]:
#Creating URL Table by inserting the URL when needed
while True:
    client = pym.MongoClient('mongodb://localhost:27017/')  #Connecting to MongoDb
    db = client["Rss_Feed"]                                 #creating db if not exist else connect
    rss_feeds_link = db["Rss_Feed_Url"]                     #creating Table if not exist else connect to the table
    
    #Asking choice to insert Data 
    print("Enter choice : \n 1. Insert \n Enter any Number to stop Inserting")
    n = int(input())
    if n == 1:
        url = input()   
        d = date.today().strftime("%a, %d %b %Y %H:%M:%S %z")            #Storing Current Date and Time
        ur = {"link":url,"lastupdate":d}                                 #Creating a query to insert url 
        rss_feeds_link.update_one(ur,{'$set':ur},upsert=True)            #Inserting or Updating in the Database
    else:
        break
    


In [ ]:
#Main RSS Feed Function
def rss_feed_schedule():
    #url=["https://news.bitcoin.com/feed/"]
    #Making Connection with mongodb
    try:
        client = pym.MongoClient('mongodb://localhost:27017/')
        #creating db if not exist else connect
        db = client["Rss_Feed"]
        
        #creating Collections
        rss_feeds_info = db["Rss_Feed_Info"]                  #Collection for information
        rss_feeds_link = db["Rss_Feed_Url"]                   #Collection for Url
        
        
        url = rss_feeds_link.find()                           #Extracting the URLs from the Database
    except:
        print("Database not Connected")
        #Iterating through each URL fetched
        
    for u in url:
        print(u)
        try:
            #Parsing Data from the URL
            resp=requests.get(u['link'])                                #Accessing the URL
            soup=BeautifulSoup(resp.content, features='xml')            #Scrapping data fromn the URL 
            items=soup.findAll('item')                                  #Storing Data fetched
            build=soup.find('lastBuildDate')                            #Accessing Last Build Date
        except:
            print("Cannot Parse the Feed")
        
        if build is not None:
            for ch in build:
                ur = {"link":u['link']} 
                print(ch)
                if (ch != u['lastupdate']):

                    ur1 ={"link":u['link'],"lastupdate":ch}
                    rss_feeds_link.update_one(ur,{'$set':ur1})            #Updating Last Build Date

                    #Iterating through each item parsed
                    for item in items:
                        a = item.link.text


                        #print(item)                                       #Testing
                        try:
                            if rss_feeds_info.find({Link:a}) == False:
                                #Downloading and parsing Feed Using Newspaper Module
                                try:

                                    article = Article(a)
                                    article.download()
                                    article.parse() 
                                    article.nlp()

                                except:
                                    print("Cannot Download the Feed or Image")

                                #Storing and processing required Information
                                try:
                                    image = article.top_image
                                    des = item.description.text
                                    TAG_RE = re.compile(r'<[^>]+>')
                                    des = re.sub(TAG_RE,'', des)
                                    des= html.unescape(des)
                                    #print(des)
                                    if des == None:
                                        des = article.summary
                                except:
                                    print("Cannot Fetch Image or Description")

                                try:
                                    #Query to store the Data
                                    qry = {"Title":article.title, "Link":article.url, "Description":des,
                                           "PubDate":article.publish_date.strftime("%a, %d %b %Y %H:%M:%S %z"),"Image":image}   
                                    qry1 = {"Link":article.url}
                                    #Update the Document if already exist otherwise Insert a new Document in the collection
                                    rss_feeds_info.update_one(qry1,{'$set':qry},upsert = True)

                                except:
                                    print("Data cannot be Inserted")
                        except:
                            print("Data Present in the database")


    time.sleep(1)


#Scheduling every 1 hr 
schedule.every(20).seconds.do(rss_feed_schedule)              

while True:
    schedule.run_pending()                                    #Schedule the function if anything is pending
    #print("Data")
    time.sleep(1)
    #print("Added")

{'_id': ObjectId('61fabdd196fadf2319878924'), 'link': 'https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml', 'lastupdate': 'Tue, 15 Feb 2022 15:29:50 +0000'}
Tue, 15 Feb 2022 15:29:50 +0000
{'_id': ObjectId('61fabde296fadf2319878928'), 'link': 'https://news.bitcoin.com/feed/', 'lastupdate': 'Tue, 15 Feb 2022 14:42:11 +0000'}
Tue, 15 Feb 2022 14:42:11 +0000
{'_id': ObjectId('6200c7f0131bc8abb32d2afb'), 'link': 'https://akeo.tech/feed/', 'lastupdate': 'Mon, 24 Jan 2022 10:53:17 +0000'}
Mon, 24 Jan 2022 10:53:17 +0000
{'_id': ObjectId('6200c7fd131bc8abb32d2b0e'), 'link': 'https://blocknewsafrica.com/feed/', 'lastupdate': 'Sun, 02 Jan 2022 00:20:30 +0000'}
Sun, 02 Jan 2022 00:20:30 +0000
{'_id': ObjectId('6200c806131bc8abb32d2b1b'), 'link': 'https://blog.ionixxtech.com/blog/feed/', 'lastupdate': 'Thu, 11 Nov 2021 10:03:34 +0000'}
Thu, 11 Nov 2021 10:03:34 +0000
{'_id': ObjectId('6200c810131bc8abb32d2b29'), 'link': 'https://www.blockedge.io/blog/feed/', 'lastupdate': 'Wed, 09 Feb

Tue, 15 Feb 2022 09:25:40 +0000
{'_id': ObjectId('6200c82e131bc8abb32d2b56'), 'link': 'https://blog.coinfund.io/feed', 'lastupdate': 'Tue, 15 Feb 2022 15:31:08 GMT'}
Tue, 15 Feb 2022 15:31:08 GMT
{'_id': ObjectId('6200c836131bc8abb32d2b64'), 'link': 'https://www.blockchainappfactory.com/blog/feed/', 'lastupdate': 'Tue, 15 Feb 2022 12:40:53 +0000'}
Tue, 15 Feb 2022 12:40:53 +0000
{'_id': ObjectId('6200c978131bc8abb32d2cbd'), 'link': 'https://101blockchains.com/feed/', 'lastupdate': 'Tue, 15 Feb 2022 07:18:21 +0000'}
Tue, 15 Feb 2022 07:18:21 +0000
{'_id': ObjectId('6200c983131bc8abb32d2ccd'), 'link': 'https://www.blockchain-council.org/feed/', 'lastupdate': 'Tue, 15 Feb 2022 11:58:21 +0000'}
Tue, 15 Feb 2022 11:58:21 +0000
{'_id': ObjectId('6200c9a9131bc8abb32d2cf8'), 'link': 'https://insidebitcoins.com/feed', 'lastupdate': 'Tue, 15 Feb 2022 14:43:03 +0000'}
Tue, 15 Feb 2022 14:43:03 +0000
{'_id': ObjectId('6200c9b2131bc8abb32d2d04'), 'link': 'https://www.ibm.com/blogs/blockchain/feed/'

Wed, 09 Feb 2022 08:59:40 +0000
{'_id': ObjectId('6200c81e131bc8abb32d2b3c'), 'link': 'https://blog.quillhash.com/feed', 'lastupdate': 'Tue, 15 Feb 2022 09:25:40 +0000'}
Tue, 15 Feb 2022 09:25:40 +0000
{'_id': ObjectId('6200c82e131bc8abb32d2b56'), 'link': 'https://blog.coinfund.io/feed', 'lastupdate': 'Tue, 15 Feb 2022 15:31:08 GMT'}
Tue, 15 Feb 2022 15:31:08 GMT
{'_id': ObjectId('6200c836131bc8abb32d2b64'), 'link': 'https://www.blockchainappfactory.com/blog/feed/', 'lastupdate': 'Tue, 15 Feb 2022 12:40:53 +0000'}
Tue, 15 Feb 2022 12:40:53 +0000
{'_id': ObjectId('6200c978131bc8abb32d2cbd'), 'link': 'https://101blockchains.com/feed/', 'lastupdate': 'Tue, 15 Feb 2022 07:18:21 +0000'}
Tue, 15 Feb 2022 07:18:21 +0000
{'_id': ObjectId('6200c983131bc8abb32d2ccd'), 'link': 'https://www.blockchain-council.org/feed/', 'lastupdate': 'Tue, 15 Feb 2022 11:58:21 +0000'}
Tue, 15 Feb 2022 11:58:21 +0000
{'_id': ObjectId('6200c9a9131bc8abb32d2cf8'), 'link': 'https://insidebitcoins.com/feed', 'lastupda

Thu, 11 Nov 2021 10:03:34 +0000
{'_id': ObjectId('6200c810131bc8abb32d2b29'), 'link': 'https://www.blockedge.io/blog/feed/', 'lastupdate': 'Wed, 09 Feb 2022 08:59:40 +0000'}
Wed, 09 Feb 2022 08:59:40 +0000
{'_id': ObjectId('6200c81e131bc8abb32d2b3c'), 'link': 'https://blog.quillhash.com/feed', 'lastupdate': 'Tue, 15 Feb 2022 09:25:40 +0000'}
Tue, 15 Feb 2022 09:25:40 +0000
{'_id': ObjectId('6200c82e131bc8abb32d2b56'), 'link': 'https://blog.coinfund.io/feed', 'lastupdate': 'Tue, 15 Feb 2022 15:31:08 GMT'}
Tue, 15 Feb 2022 15:31:08 GMT
{'_id': ObjectId('6200c836131bc8abb32d2b64'), 'link': 'https://www.blockchainappfactory.com/blog/feed/', 'lastupdate': 'Tue, 15 Feb 2022 12:40:53 +0000'}
Tue, 15 Feb 2022 12:40:53 +0000
{'_id': ObjectId('6200c978131bc8abb32d2cbd'), 'link': 'https://101blockchains.com/feed/', 'lastupdate': 'Tue, 15 Feb 2022 07:18:21 +0000'}
Tue, 15 Feb 2022 07:18:21 +0000
{'_id': ObjectId('6200c983131bc8abb32d2ccd'), 'link': 'https://www.blockchain-council.org/feed/', 'last

Mon, 24 Jan 2022 10:53:17 +0000
{'_id': ObjectId('6200c7fd131bc8abb32d2b0e'), 'link': 'https://blocknewsafrica.com/feed/', 'lastupdate': 'Sun, 02 Jan 2022 00:20:30 +0000'}
Sun, 02 Jan 2022 00:20:30 +0000
{'_id': ObjectId('6200c806131bc8abb32d2b1b'), 'link': 'https://blog.ionixxtech.com/blog/feed/', 'lastupdate': 'Thu, 11 Nov 2021 10:03:34 +0000'}
Thu, 11 Nov 2021 10:03:34 +0000
{'_id': ObjectId('6200c810131bc8abb32d2b29'), 'link': 'https://www.blockedge.io/blog/feed/', 'lastupdate': 'Wed, 09 Feb 2022 08:59:40 +0000'}
Wed, 09 Feb 2022 08:59:40 +0000
{'_id': ObjectId('6200c81e131bc8abb32d2b3c'), 'link': 'https://blog.quillhash.com/feed', 'lastupdate': 'Tue, 15 Feb 2022 09:25:40 +0000'}
Tue, 15 Feb 2022 09:25:40 +0000
{'_id': ObjectId('6200c82e131bc8abb32d2b56'), 'link': 'https://blog.coinfund.io/feed', 'lastupdate': 'Tue, 15 Feb 2022 15:31:08 GMT'}
Tue, 15 Feb 2022 15:31:08 GMT
{'_id': ObjectId('6200c836131bc8abb32d2b64'), 'link': 'https://www.blockchainappfactory.com/blog/feed/', 'lastu

...

{'_id': ObjectId('62055f83f8061bb9786aa188'), 'link': 'https://dappradar.com/blog/feed', 'lastupdate': 'Fri, 11 Feb 2022 00:00:00 '}
{'_id': ObjectId('620642cafb0fcc9b7cf1987a'), 'lastupdate': 'Fri, 11 Feb 2022 00:00:00 ', 'link': 'https://blockchain.news/RSS/'}
{'_id': ObjectId('61fabdd196fadf2319878924'), 'link': 'https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml', 'lastupdate': 'Tue, 15 Feb 2022 16:04:49 +0000'}
Tue, 15 Feb 2022 16:04:49 +0000
{'_id': ObjectId('61fabde296fadf2319878928'), 'link': 'https://news.bitcoin.com/feed/', 'lastupdate': 'Tue, 15 Feb 2022 14:42:11 +0000'}
Tue, 15 Feb 2022 14:42:11 +0000
{'_id': ObjectId('6200c7f0131bc8abb32d2afb'), 'link': 'https://akeo.tech/feed/', 'lastupdate': 'Mon, 24 Jan 2022 10:53:17 +0000'}
Mon, 24 Jan 2022 10:53:17 +0000
{'_id': ObjectId('6200c7fd131bc8abb32d2b0e'), 'link': 'https://blocknewsafrica.com/feed/', 'lastupdate': 'Sun, 02 Jan 2022 00:20:30 +0000'}
Sun, 02 Jan 2022 00:20:30 +0000
{'_id': ObjectId('6200c806131bc8a

Thu, 11 Nov 2021 10:03:34 +0000
{'_id': ObjectId('6200c810131bc8abb32d2b29'), 'link': 'https://www.blockedge.io/blog/feed/', 'lastupdate': 'Wed, 09 Feb 2022 08:59:40 +0000'}
Wed, 09 Feb 2022 08:59:40 +0000
{'_id': ObjectId('6200c81e131bc8abb32d2b3c'), 'link': 'https://blog.quillhash.com/feed', 'lastupdate': 'Tue, 15 Feb 2022 09:25:40 +0000'}
Tue, 15 Feb 2022 09:25:40 +0000
{'_id': ObjectId('6200c82e131bc8abb32d2b56'), 'link': 'https://blog.coinfund.io/feed', 'lastupdate': 'Tue, 15 Feb 2022 16:01:45 GMT'}
Tue, 15 Feb 2022 16:01:45 GMT
{'_id': ObjectId('6200c836131bc8abb32d2b64'), 'link': 'https://www.blockchainappfactory.com/blog/feed/', 'lastupdate': 'Tue, 15 Feb 2022 12:40:53 +0000'}
Tue, 15 Feb 2022 12:40:53 +0000
{'_id': ObjectId('6200c978131bc8abb32d2cbd'), 'link': 'https://101blockchains.com/feed/', 'lastupdate': 'Tue, 15 Feb 2022 07:18:21 +0000'}
Tue, 15 Feb 2022 07:18:21 +0000
{'_id': ObjectId('6200c983131bc8abb32d2ccd'), 'link': 'https://www.blockchain-council.org/feed/', 'last

Tue, 15 Feb 2022 15:48:54 +0000
{'_id': ObjectId('6200c9b2131bc8abb32d2d04'), 'link': 'https://www.ibm.com/blogs/blockchain/feed/', 'lastupdate': 'Fri, 04 Feb 2022 20:28:29 +0000'}
Fri, 04 Feb 2022 20:28:29 +0000
{'_id': ObjectId('6200c9be131bc8abb32d2d1a'), 'link': 'https://www.btcwires.com/feed/', 'lastupdate': 'Wed, 09 Feb 2022 14:02:50 +0000'}
Wed, 09 Feb 2022 14:02:50 +0000
{'_id': ObjectId('62055cd8f8061bb9786a9de2'), 'link': 'https://immunebytes.com/feed/', 'lastupdate': 'Fri, 11 Feb 2022 00:00:00 '}
{'_id': ObjectId('62055f83f8061bb9786aa188'), 'link': 'https://dappradar.com/blog/feed', 'lastupdate': 'Fri, 11 Feb 2022 00:00:00 '}
{'_id': ObjectId('620642cafb0fcc9b7cf1987a'), 'lastupdate': 'Fri, 11 Feb 2022 00:00:00 ', 'link': 'https://blockchain.news/RSS/'}
{'_id': ObjectId('61fabdd196fadf2319878924'), 'link': 'https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml', 'lastupdate': 'Tue, 15 Feb 2022 16:11:13 +0000'}
Tue, 15 Feb 2022 16:11:13 +0000
{'_id': ObjectId('61fab

Tue, 15 Feb 2022 14:42:11 +0000
{'_id': ObjectId('6200c7f0131bc8abb32d2afb'), 'link': 'https://akeo.tech/feed/', 'lastupdate': 'Mon, 24 Jan 2022 10:53:17 +0000'}
Mon, 24 Jan 2022 10:53:17 +0000
{'_id': ObjectId('6200c7fd131bc8abb32d2b0e'), 'link': 'https://blocknewsafrica.com/feed/', 'lastupdate': 'Sun, 02 Jan 2022 00:20:30 +0000'}
Sun, 02 Jan 2022 00:20:30 +0000
{'_id': ObjectId('6200c806131bc8abb32d2b1b'), 'link': 'https://blog.ionixxtech.com/blog/feed/', 'lastupdate': 'Thu, 11 Nov 2021 10:03:34 +0000'}
Thu, 11 Nov 2021 10:03:34 +0000
{'_id': ObjectId('6200c810131bc8abb32d2b29'), 'link': 'https://www.blockedge.io/blog/feed/', 'lastupdate': 'Wed, 09 Feb 2022 08:59:40 +0000'}
Wed, 09 Feb 2022 08:59:40 +0000
{'_id': ObjectId('6200c81e131bc8abb32d2b3c'), 'link': 'https://blog.quillhash.com/feed', 'lastupdate': 'Tue, 15 Feb 2022 09:25:40 +0000'}
Tue, 15 Feb 2022 09:25:40 +0000
{'_id': ObjectId('6200c82e131bc8abb32d2b56'), 'link': 'https://blog.coinfund.io/feed', 'lastupdate': 'Tue, 15 Feb

Tue, 15 Feb 2022 09:25:40 +0000
{'_id': ObjectId('6200c82e131bc8abb32d2b56'), 'link': 'https://blog.coinfund.io/feed', 'lastupdate': 'Tue, 15 Feb 2022 16:01:45 GMT'}
Tue, 15 Feb 2022 18:55:01 GMT
Data Present in the database
Data Present in the database
Data Present in the database
Data Present in the database
Data Present in the database
Data Present in the database
Data Present in the database
Data Present in the database
Data Present in the database
Data Present in the database
{'_id': ObjectId('6200c836131bc8abb32d2b64'), 'link': 'https://www.blockchainappfactory.com/blog/feed/', 'lastupdate': 'Tue, 15 Feb 2022 12:40:53 +0000'}
Tue, 15 Feb 2022 12:40:53 +0000
{'_id': ObjectId('6200c978131bc8abb32d2cbd'), 'link': 'https://101blockchains.com/feed/', 'lastupdate': 'Tue, 15 Feb 2022 07:18:21 +0000'}
Tue, 15 Feb 2022 07:18:21 +0000
{'_id': ObjectId('6200c983131bc8abb32d2ccd'), 'link': 'https://www.blockchain-council.org/feed/', 'lastupdate': 'Tue, 15 Feb 2022 11:58:21 +0000'}
Tue, 15 Fe

In [ ]:
rss_feed_schedule()
